In [130]:
import pandas as pd
import numpy as np
import scipy
import scipy.stats
from keras.models import Sequential, load_model, Model 
from keras.layers import Dense, Activation, Flatten, Dropout, Input, BatchNormalization, Activation, add, MaxPooling1D, Cropping1D 
from keras.layers.convolutional import Conv1D
from keras.optimizers import Adam
#from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
#import hyperopt.fmin as hypfmin
import keras
import tensorflow as tf
import random
import os
import pickle
import re
from math import ceil
from Bio import SeqIO
import warnings 
import datetime
import keras.backend as kb
from sklearn.metrics import average_precision_score, precision_recall_curve
import h5py
from keras.layers.merge import concatenate
from keras.layers.core import Lambda


In [131]:
#select transcripts that are protein coding, principal isoforms, with splice sites 
def processGff(fn):
    GCDE = pd.read_csv(fn,sep='\t',comment='#',header=None)
    pcgBool=[bool(re.search("gene_type=protein_coding",s)) for s in GCDE[8]]
    GCDE=GCDE[pcgBool]
    GCDE['trx_name']=GCDE.iloc[:,8].str.extract("transcript_name=(.+?);")
    GCDE['trx_id']=GCDE.iloc[:,8].str.extract("transcript_id=(.+?);")
    GCDE['gene']=GCDE.iloc[:,8].str.extract('gene_name=(.+?);')
    trxts, exons = GCDE[GCDE[2]=="transcript"], GCDE[GCDE[2]=="exon"]
    princIso=[bool(re.search("appris_principal",s)) for s in trxts[8]]
    trxts=trxts[princIso]
    trxts['tsl']=trxts.iloc[:,8].str.extract('transcript_support_level=(.+?);')
    trxts['level']=trxts.iloc[:,8].str.extract(';level=(.+?);')
    trxts['alt']=[bool(re.search(';tag=.*?(alternative).*?;',s)) for s in trxts[8]]
    trxts['trxSz']=trxts[4]-trxts[3]
    trxts['tsl']=pd.to_numeric(trxts['tsl'], errors='coerce')
    trxts['level']=pd.to_numeric(trxts['level'], errors='coerce')
    splitGps = trxts.groupby('gene')
    dups=pd.DataFrame()
    trxts=pd.DataFrame()
    for sg in splitGps:
        sg=sg[1]
        if len(sg) > 1:
            sg=sg[sg['tsl'] == sg['tsl'].min()]
            if len(sg) > 1:
                sg=sg[sg['level'] == sg['level'].min()]
                if len(sg) > 1:
                    sg=sg[~sg['alt'].values]
                    if len(sg) > 1:
                        sg=sg[sg['trxSz'] == sg['trxSz'].max()]
                        if len(sg) > 1:
                            dups=dups.append(sg)
                            continue 
        trxts=trxts.append(sg)
    if len(dups) > 0:
        print("{} Removed (Still has duplicates!!!)".format(dups['trx_name']))
        print("{} gene(s) with duplicates removed!!!".format(len(set(dups['gene']))))
    exons=exons[exons["trx_id"].isin(trxts['trx_id'])]
    loneExns=exons.drop_duplicates(subset='trx_id',keep=False).loc[:,"trx_id"].values
    trxts=trxts[~trxts["trx_id"].isin(loneExns)]
    exons=exons[~exons["trx_id"].isin(loneExns)]
    cols=list(range(0,9))+['trx_name','gene']
    trxts=trxts.loc[:,cols]
    exons=exons.loc[:,cols]
    return trxts,exons



In [132]:
#remove any genes with pseudogenes
def rmPseudoGns(tstTrxts,tstExons,pgsFile):
    pgs=pd.read_csv(pgsFile,comment='#')
    pgs.dropna(inplace=True)
    pgsNms=set(pgs['Human paralogue associated gene name'])
    tstTrxts=tstTrxts[~tstTrxts.loc[:,'gene'].isin(pgsNms)]
    tstExons=tstExons[~tstExons.loc[:,'gene'].isin(pgsNms)]
    return tstTrxts,tstExons

In [133]:
#which chromosomes to subset from transcripts and corresponding features 
def subChroms(trxts,exons,chrs):
    chrs=['chr'+str(c) for c in chrs]
    trxts=trxts[trxts[0].isin(chrs)].sort_values(0)
    exons=exons[exons[0].isin(chrs)]
    return trxts,exons

In [154]:
#target matrix one-hot encoded for 3 classes: SA,SD,and neither 
def getTargetMtxs(trxts,InpMtxs,exons):
    warnings.filterwarnings("ignore")
    tgtMtxs=[]
    for r in range(len(trxts)):
        row=trxts.iloc[r,:]
        trxtExs=exons[exons['trx_name']==row['trx_name']]
        trxtExs.sort_values(3,inplace=True)
        trxtW=row[4]-row[3]+1
        ons=np.matrix(np.ones(trxtW))
        zs=np.matrix(np.zeros((2,trxtW)))
        tgtMtx=np.concatenate((ons,zs))
        if row[6] == '+':
            exStarts, exEnds = list(trxtExs[3])[1:], list(trxtExs[4])[:-1]
            exStarts=[eS - row[3] for eS in exStarts]
            exEnds=[eE - row[3] for eE in exEnds]
        else:
            exStarts, exEnds = list(trxtExs[4])[:-1], list(trxtExs[3])[1:]
            exStarts=[eS - row[3] for eS in exStarts]
            exEnds=[eE - row[3] for eE in exEnds]
        for eS in exStarts:
            try:
                tgtMtx[1,eS] = 1
                tgtMtx[0,eS] = 0
            except:
                print(row)
        for eE in exEnds:
            try:
                tgtMtx[2,eE] = 1
                tgtMtx[0,eE] = 0
            except:
                return row
        if row[6] == '-' :
            tgtMtx = np.fliplr(tgtMtx)
        InMx = np.array(InpMtxs[r])
        Nis = [Ni for Ni in range(InMx.shape[1]) if (InMx[:,Ni] == [[0],[0],[0],[0]]).all()]
        if len(Nis)>0:
            for Ni in Nis:
                tgtMtx[:,Ni]=[[0],[0],[0]]   
        tgtMtxs.append(tgtMtx)
    return tgtMtxs
     
    

In [135]:
#get transcript sequences into one-hot encoded form 
def getInputMtxs(genomeFasta,trxts):
    warnings.filterwarnings("ignore")
    fasta_sequences = SeqIO.parse(open(genomeFasta),'fasta')
    seqs={}
    for seq in fasta_sequences:
        if seq.name[:3]=='chr':
            seqs[seq.name]=seq  
    trnMtxs=[]
    posBases=dict(zip(['A','C','G','T'],range(4)))
    negBases=dict(zip(['T','G','C','A'],range(4)))
    for r in range(len(trxts)):
        row=trxts.iloc[r,:]
        trxtW=row[4]-row[3]+1
        trxtSeq=seqs[row[0]].seq[row[3]-1:row[4]]
        trnMtx = np.matrix(np.zeros((4,trxtW)))
        if row[6] == '+':
            for bi in range(trxtW):
                try:
                    b = trxtSeq[bi]
                    trnMtx[posBases[b],bi] = 1
                except:
                    continue 
        else:
            for bi in range(trxtW):        
                try:
                    b = trxtSeq[bi]            
                    trnMtx[negBases[b],bi] = 1
                except:
                    continue 
            trnMtx = np.fliplr(trnMtx)
        trnMtxs.append(trnMtx)
    return trnMtxs


In [136]:
# S/2 + I + S/2 sized blocks to feed into model where S is overall size of flank and I is sequences to predict status 
def getInputBlocks(trnMtxs,flankL):
    trnBlocks=[]
    halfFlankL=int(flankL/2)
    for trnMtx in trnMtxs:
        numBlocks = ceil(len(trnMtx.T)/5000)
        blW = numBlocks * 5000
        bAdd = blW - len(trnMtx.T) 
        rightPad=int(halfFlankL + bAdd)
        trnMtxZpd = np.pad(trnMtx,((0,0),(halfFlankL,rightPad)),'constant')
        for bln in range(numBlocks):
            blLow = 5000*(bln) 
            blHigh = 5000*(bln+1) + flankL # - 1 in blHigh is omitted  
            block = trnMtxZpd[:,blLow:blHigh]
            trnBlocks.append(block)
    return trnBlocks
    

In [137]:
# target blocks in size I blocks 
def getTargetBlocks(tgtMtxs):
    tgtBlocks=[]
    for tgtMtx in tgtMtxs:
        numBlocks = ceil(len(tgtMtx.T)/5000)
        blW = numBlocks * 5000
        bAdd = blW - len(tgtMtx.T) 
        tgtMtxZpd = np.pad(tgtMtx,((0,0),(0,bAdd)),'constant')
        for bln in range(numBlocks):
            blLow = 5000*(bln) 
            blHigh = 5000*(bln+1) # - 1 in blHigh is omitted  
            block = tgtMtxZpd[:,blLow:blHigh]
            tgtBlocks.append(block)
    return tgtBlocks

In [6]:
def Mtxs_toH5Blocks(InpMtxs,TgtMtxs,flankL,CHUNK_SIZE,train_or_test):
    #assert len(InpMtxs)==len(TgtMtxs)
    num_CHUNKS=ceil(len(InpMtxs)/CHUNK_SIZE)
    h5f_Inp=h5py.File('Inp_'+train_or_test+'.h5', 'w')
    h5f_Tgt=h5py.File('Tgt_'+train_or_test+'.h5', 'w')
    for i in range(num_CHUNKS):
        Ib=getInputBlocks(InpMtxs[CHUNK_SIZE*i:CHUNK_SIZE*(i+1)],flankL)
        Tb=getTargetBlocks(TgtMtxs[CHUNK_SIZE*i:CHUNK_SIZE*(i+1)])
        h5f_Inp[str(i)]=np.swapaxes(np.stack(Ib,axis=1).T,0,1).astype('int8')
        h5f_Tgt[str(i)]=np.swapaxes(np.stack(Tb,axis=1).T,0,1).astype('int8')
    print ('Inp_'+train_or_test+'.h5'+'and'+'Tgt_'+train_or_test+'.h5'+'created')
            

In [141]:
#create multiple resnet layers in model 
def resnet(x,fltrNumb,fltrW,dilr,numbRns):
    def oneLoop(x):
        z = BatchNormalization()(x)
        z = Activation('relu')(z)
        z = Conv1D(fltrNumb,kernel_size=(fltrW,),dilation_rate=dilr,padding='same')(z)
        return z
    full=x
    for n in range(numbRns):
        z=oneLoop(full)
        z=oneLoop(z)
        full = add([full,z]) 
    return full   

In [ ]:
# Standard categorical cross entropy for sequence outputs
def categorical_crossentropy_2d(y_true, y_pred):
    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10)
                   + y_true[:, :, 2]*kb.log(y_pred[:, :, 2]+1e-10))

In [ ]:
def make_parallel(model, gpu_count):

    def get_slice(data, idx, parts):

        shape = tf.shape(data)
        stride = tf.concat([shape[:1]//parts, shape[1:]*0], 0)
        start = stride * idx

        size = tf.concat([shape[:1]//parts, shape[1:]], 0) 
        # Split the batch into equal parts 

        return tf.slice(data, start, size)

    outputs_all = []
    for i in range(len(model.outputs)):
        outputs_all.append([])

    # Place a copy of the model on each GPU, each getting a slice of the batch
    for i in range(gpu_count):
        with tf.device('/gpu:%d' % i):
            with tf.name_scope('tower_%d' % i) as scope:

                inputs = []
                # Slice each input into a piece for processing on this GPU
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_n = Lambda(get_slice, output_shape=input_shape,
                                  arguments={'idx': i, 'parts': gpu_count})(x)
                    inputs.append(slice_n)

                outputs = model(inputs)
                
                if not isinstance(outputs, list):
                    outputs = [outputs]
                
                # Save all the outputs for merging back together later
                for l in range(len(outputs)):
                    outputs_all[l].append(outputs[l])

    # Merge outputs on CPU
    with tf.device('/cpu:0'):
        
        merged = []
        for outputs in outputs_all:
            merged.append(concatenate(outputs, axis=0))
            
        return Model(inputs=model.inputs, outputs=merged)


In [ ]:
def topK_auPRC(model,Inp,Tgt):
    p=model.predict(Inp)
    Tks=[]
    PRs=[]
    for c in [1,2]:
        Fullc=Tgt[:,:,c].flatten()
        S=[i for i in range(len(Fullc)) if Fullc[i] == 1]
        pS=np.argsort(p[:,:,c].flatten())[-len(S):]
        pS=pd.Series(pS)
        nCrct=pS[pS.isin(S)].size
        Tk=nCrct/len(S)
        Tks.append(Tk)
        PR=average_precision_score(Fullc,p[:,:,c].flatten())
        PRs.append(PR)
    return(Tks[0],Tks[1],PRs[0],PRs[1])